In [50]:
#### Package import ####
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import os
import cv2
import numpy as np
import scipy.io as sio
from PIL import Image

In [58]:
class TVNet(object):
    GRAD_IS_ZERO = 1e-12

    def __init__(self):
        pass

    def grey_scale_image(self, x):
        imageTransform = torchvision.transforms.ToPILImage()
        tensorTransform = torchvision.transforms.ToTensor()
        greyScaleTransform = torchvision.transforms.Grayscale(num_output_channels=1)
        grey_x = greyScaleTransform(x)
        
        # return tf.floor(grey_x * 255)
        
        return torch.floor(tensorTransform(grey_x) * 255)
    
    def normalize_img_tensor(self, x):
        return (x - torch.min(x)) / (torch.max(x) - torch.min(x))

    def gaussian_smooth(self, x):
        nb_channels = 1
        
        h, w = x.size()
        y = torch.ones(1, nb_channels, h, w)
        y[:, :, :, :] = x;
        weights = torch.tensor([[0.000874, 0.006976, 0.01386, 0.006976, 0.000874],
                        [0.006976, 0.0557, 0.110656, 0.0557, 0.006976],
                        [0.01386, 0.110656, 0.219833, 0.110656, 0.01386],
                        [0.006976, 0.0557, 0.110656, 0.0557, 0.006976],
                        [0.000874, 0.006976, 0.01386, 0.006976, 0.000874]])
        weights = weights.view(1, nb_channels, 5, 5).repeat(1, nb_channels, 1, 1)

        return torch.squeeze(F.conv2d(y, weights))
        
    '''
    def warp_image(self, x, u, v):
        #assert len(x.shape) == 4
        #assert len(u.shape) == 3
        #assert len(v.shape) == 3

        # rescale the unit to be pixel
        # u = u / x.shape[2].value * 2
        # v = v / x.shape[1].value * 2
        u = u / x.size()[2] * 2
        v = v / x.size()[1] * 2

        # delta = tf.concat(axis=1, values=[u, v])
        delta = torch.cat([u, v],dim=1)
        # return spatial_transformer.transformer(x, delta, (x.shape[-3].value, x.shape[-2].value))
    '''    
    
    
    def centered_gradient(self, img):
        nx, ny = img.size()
        
        dy = torch.zeros(img.size())
        
        for i in range(1, nx - 1):
            dy[i,:] = img[i+1,:] -  img[i-1,:]
        dy[0,:] = img[1,:] - img[0,:]
        dy[ny - 1,:] = img[ny-1,:] - img[ny-2,:]

        dx = torch.zeros(img.size())
        
        for i in range(1, ny - 1):
            dx[:,i] = img[:,i+1] - img[:,i-1]
        dx[:,0] = img[:,1] - img[:,0]
        dx[:, ny - 1] = img[:, ny - 1] - img[:, ny - 2]

        dx = dx * .5
        dy = dy * .5

        return dx, dy
    
    def zoom_size(self, height, width, factor):
        new_height = int(float(height) * factor + 0.5)
        new_width = int(float(width) * factor + 0.5)

        return new_height, new_width

    def zoom_image(self, x, new_height, new_width):
        # assert len(x.shape) == 4
        
        delta = torch.zeros(x.size()[0], 2, new_height * new_width)
        # delta = tf.zeros((tf.shape(x)[0], 2, new_height * new_width))
        # zoomed_x = spatial_transformer.transformer(x, delta, (new_height, new_width))
        # return tf.reshape(zoomed_x, [tf.shape(x)[0], new_height, new_width, x.shape[-1].value])

In [60]:
# load image
img1 = Image.open("/home/truppr/sandbox/tvnet/testFrames/toys1.png")
print(img1.size)

# start TVNet model
T = TVNet()

# convert to greyscale and normalize image
img1 = torch.squeeze(T.normalize_img_tensor(T.grey_scale_image(img1)))
print(img1)
print(img1.size())

# apply gaussian smoothing
img1 = T.gaussian_smooth(img1)
print(img1)
print(img1.size())

#u_flat = tf.reshape(u1, (tf.shape(x2)[0], 1, x2.shape[1].value * x2.shape[2].value))
#v_flat = tf.reshape(u2, (tf.shape(x2)[0], 1, x2.shape[1].value * x2.shape[2].value))
# warped = T.warp_image(img1,u_flat,v_flat)

# computer centered gradient
img1_dx, img1_dy = T.centered_gradient(img1)
print(img1_dx)
print(img1_dy)

(256, 256)
tensor([[0.0451, 0.0451, 0.0533,  ..., 0.0656, 0.0574, 0.0574],
        [0.0492, 0.0451, 0.0205,  ..., 0.0574, 0.0533, 0.0574],
        [0.0533, 0.0533, 0.0533,  ..., 0.0533, 0.0533, 0.0574],
        ...,
        [0.5205, 0.5369, 0.5369,  ..., 0.0287, 0.0287, 0.0492],
        [0.5164, 0.5369, 0.5164,  ..., 0.0287, 0.0328, 0.0369],
        [0.5164, 0.5451, 0.5369,  ..., 0.0369, 0.5164, 0.5451]])
torch.Size([256, 256])
tensor([[0.0448, 0.0488, 0.0523,  ..., 0.0569, 0.0567, 0.0555],
        [0.0427, 0.0475, 0.0522,  ..., 0.0576, 0.0576, 0.0555],
        [0.0417, 0.0466, 0.0517,  ..., 0.0575, 0.0575, 0.0544],
        ...,
        [0.5514, 0.5647, 0.5776,  ..., 0.0398, 0.0418, 0.0466],
        [0.5388, 0.5526, 0.5648,  ..., 0.0341, 0.0349, 0.0374],
        [0.5350, 0.5454, 0.5531,  ..., 0.0312, 0.0313, 0.0353]])
torch.Size([252, 252])
tensor([[ 0.0020,  0.0038,  0.0024,  ...,  0.0001, -0.0007, -0.0006],
        [ 0.0024,  0.0047,  0.0030,  ...,  0.0004, -0.0011, -0.0010],
       